In [ ]:
## module error
''' err(string).
    Prints 'string' and terminates program.
'''
import sys
def err(string):
    print(string)
    raw_input('Press return to exit')
    sys.exit()

In [ ]:
## module gaussPivot
''' x = gaussPivot(a,b,tol=1.0e-9).
    Memecahkan [a]{x} = {b} dengan Eliminasi Gauss
    dan pivoting baris
'''

from numpy import*
import swap
import error

def gaussPivot(a,b,tol=1.0e-12):
    n = len(b)

    # Setup faktor skala
    s = zeros((n))
    for i in range(n):
        s[i] = max(abs(a[i,:]))

    for k in range(0,n-1):

        #Perubahan baris jika diperlukan
        p = int(argmax(abs(a[k:n,k])/s[k:n])) + k
        if abs(a[p,k]) < tol: error.err('Matriks adalah singular')
        if p != k:
            swap.swapRows(b,k,p)
            swap.swapRows(s,k,p)
            swap.swapRows(a,k,p)

        # Eliminasi
        for i in range(k+1,n):
            if a[i,k] != 0.0:
                lam = a[i,k]/a[k,k]
                a[i,k+1:n] = a[i,k+1:n] - lam*a[k,k+1:n]
                b[i] = b[i] - lam*b[k]
        if abs(a[n-1,n-1]) < tol: error.err('Matriks adalah singular')

    # Substitusi balik
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - dot(a[k,k+1:n],b[k+1:n]))/a[k,k]
    return b


In [ ]:
## module polyFit
''' c = polyFit(xData,yData,m).
    Menentukan nilai koefisien polynomial
    p(x) = c[0] + c[1]x + c[2]x^2 +...+ c[m]x^m
    yang memfitting data dengan metode least squares.

    sigma = stdDev(c,xDara,yData).
    Menghitung std. deviasi antara p(x) dan data.
'''

from numpy import zeros
from math import sqrt
from gaussPivot import*

def polyFit(xData,yData,m):
    a = zeros((m+1,m+1))
    b = zeros((m+1))
    s = zeros((2*m+1))
    for i in range(len(xData)):
        temp = yData[i]
        for j in range(m+1):
            b[j] = b[j] + temp
            temp = temp*xData[i]
        temp = 1.0
        for j in range(2*m+1):
            s[j] = s[j] + temp
            temp = temp*xData[i]
    for i in range(m+1):
        for j in range(m+1):
            a[i,j] = s[i+j]
    return gaussPivot(a,b)

def stdDev(c,xData,yData):

    def evalPoly(c,x):
        m = len(c) - 1
        p = c[m]
        for j in range(m):
            p = p*x + c[m-j-1]
        return p

    n = len(xData) - 1
    m = len(c) - 1
    sigma = 0.0
    for i in range(n+1):
        p = evalPoly(c,xData[i])
        sigma = sigma + (yData[i] - p)**2
    sigma = sqrt(sigma/(n - m))
    return sigma

In [ ]:
def swapRows(v,i,j):
    if len(v.shape) == 1:
        v[i],v[j] = v[j],v[i]
    else:
        v[[i,j],:] = v[[j,i],:]
def swapCols(v,i,j):
    v[:,[i,j]] = v[:,[j,i]]

In [ ]:
import numpy as np
from polyFit import*
import matplotlib.pyplot as plt
#input data
xData = np.array([0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0])
yData = np.array([20.0,40.0,60.0,80.0,100.0,120.0,140.0,160.0,180.0])
while True:
    try:
        m = eval(input("\nPolinomial Orde ==> "))
        coeff = polyFit(xData,yData,m)
        print("koefisien:\n",coeff)
        print("Std. deviasi =",stdDev(coeff,xData,yData))
        m = len(coeff)
        x1 = min(xData)
        x2 = max(xData)
        dx = (x2 - x1)/20.0
        x = np.arange(x1,x2 + dx/10.0,dx)
        y = np.zeros((len(x)))*1.0
        for i in range(m):
            y = y + coeff[i]*x**i
        plt.plot(xData,yData,'o',x,y,'-')
        plt.title('Grafik Kecepatan vs Waktu')
        plt.xlabel('Waktu (s)')
        plt.ylabel('Kecepatan (m/s)')
        plt.legend(('Data','Regresi Polinom'),loc = 0)
        plt.grid(True)
        plt.show()
    except SyntaxError: break
input("Finished. Press return to exit")